## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lugovoy,KZ,2022-07-26 20:28:33,42.9472,72.7644,78.87,26,5,4.50,clear sky
1,1,Hailar,CN,2022-07-26 20:28:33,49.2000,119.7000,62.71,92,0,8.19,clear sky
2,2,Mokhsogollokh,RU,2022-07-26 20:28:34,61.4681,128.9203,59.63,79,69,1.45,broken clouds
3,3,Camacha,PT,2022-07-26 20:28:34,33.0833,-16.3333,67.42,88,40,14.97,scattered clouds
4,4,Iraray,PH,2022-07-26 20:28:35,8.9936,118.0511,79.20,83,33,5.77,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the lowest temperature you would like for your trip? "))
max_temp = float(input("What is the highest temperature you would like for your trip? "))

What is the lowest temperature you would like for your trip? 75
What is the highest temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lugovoy,KZ,2022-07-26 20:28:33,42.9472,72.7644,78.87,26,5,4.50,clear sky
4,4,Iraray,PH,2022-07-26 20:28:35,8.9936,118.0511,79.20,83,33,5.77,scattered clouds
6,6,Ahuimanu,US,2022-07-26 20:28:37,21.4447,-157.8378,80.31,74,75,16.11,broken clouds
9,9,Hambantota,LK,2022-07-26 20:28:39,6.1241,81.1185,78.55,84,95,2.42,overcast clouds
11,11,Hithadhoo,MV,2022-07-26 20:28:40,-0.6000,73.0833,81.01,76,100,13.29,light rain


In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

250


City_ID                250
City                   250
Country                250
Date                   250
Lat                    250
Lng                    250
Max Temp               250
Humidity               250
Cloudiness             250
Wind Speed             250
Current Description    250
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis='index', how="any")
print(len(clean_df))
clean_df.head()

250


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lugovoy,KZ,2022-07-26 20:28:33,42.9472,72.7644,78.87,26,5,4.50,clear sky
4,4,Iraray,PH,2022-07-26 20:28:35,8.9936,118.0511,79.20,83,33,5.77,scattered clouds
6,6,Ahuimanu,US,2022-07-26 20:28:37,21.4447,-157.8378,80.31,74,75,16.11,broken clouds
9,9,Hambantota,LK,2022-07-26 20:28:39,6.1241,81.1185,78.55,84,95,2.42,overcast clouds
11,11,Hithadhoo,MV,2022-07-26 20:28:40,-0.6000,73.0833,81.01,76,100,13.29,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lugovoy,KZ,78.87,clear sky,42.9472,72.7644,
4,Iraray,PH,79.20,scattered clouds,8.9936,118.0511,
6,Ahuimanu,US,80.31,broken clouds,21.4447,-157.8378,
9,Hambantota,LK,78.55,overcast clouds,6.1241,81.1185,
11,Hithadhoo,MV,81.01,light rain,-0.6000,73.0833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json' 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        continue 
        

In [9]:

# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis='index', how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Lugovoy,KZ,78.87,clear sky,42.9472,72.7644,Ж.Д Вокзал Луговая
4,Iraray,PH,79.20,scattered clouds,8.9936,118.0511,
6,Ahuimanu,US,80.31,broken clouds,21.4447,-157.8378,Paradise Bay Resort
9,Hambantota,LK,78.55,overcast clouds,6.1241,81.1185,Bungalow 63
11,Hithadhoo,MV,81.01,light rain,-0.6000,73.0833,Scoop Guest House
12,Jamestown,US,79.52,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
13,Vila Franca Do Campo,PT,79.20,scattered clouds,37.7167,-25.4333,Pestana Bahia Praia
14,Puerto Escondido,MX,88.63,scattered clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
15,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
17,Celestun,MX,85.26,scattered clouds,20.8667,-90.4000,Posada Lilia


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.3)

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

# 11b. Display the figure

fig

Figure(layout=FigureLayout(height='420px'))